In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
         .master("local[*]") \
         .appName('test') \
         .getOrCreate()

24/03/03 19:57:33 WARN Utils: Your hostname, macmint resolves to a loopback address: 127.0.1.1; using 192.168.50.165 instead (on interface enp2s0)
24/03/03 19:57:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/03 19:57:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.options(header="true", inferSchema="true").csv('data/raw/green/*/*/*.csv.gz')

In [3]:
df_green.createOrReplaceTempView('green')

In [5]:
df_green_revenue = spark.sql("""
SELECT
    date_trunc('month', lpep_pickup_datetime) AS hour,
    PULocationID AS zone,
    

    SUM(total_amount) AS amount,

    -- # records in each zone
    COUNT(1) AS number_records
FROM
    green
WHERE
    -- filter out data not from 2020
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [6]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 00:00:00| 258|12992.799999999985|           422|
|2020-02-01 00:00:00|  12| 73.74000000000001|             2|
|2020-03-01 00:00:00| 202| 718.6999999999998|            35|
|2020-01-01 00:00:00| 191|26858.520000000084|           782|
|2020-02-01 00:00:00| 188|52174.299999999195|          2115|
|2020-03-01 00:00:00|   7| 93256.04000000852|          7411|
|2020-03-01 00:00:00| 232|2700.1500000000005|            94|
|2020-01-01 00:00:00|  59| 806.6000000000001|            32|
|2020-01-01 00:00:00|  36| 24738.00999999994|          1001|
|2020-01-01 00:00:00| 171|16140.340000000006|           501|
|2020-02-01 00:00:00|  41| 288144.1599999318|         23040|
|2020-02-01 00:00:00|  15| 5257.900000000001|           149|
|2020-03-01 00:00:00| 197| 28972.29999999987|           846|
|2020-03-01 00:00:00| 12

In [7]:
df_green_revenue.repartition(20).write.parquet('data/report/revenue/green', mode='overwrite')

In [8]:
df_yellow = spark.read.options(header="true", inferSchema="true").csv('data/raw/yellow/*/*/*.csv.gz')

In [9]:
df_yellow.createOrReplaceTempView('yellow')

In [10]:
df_yellow.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge']

In [11]:
df_yellow_revenue = spark.sql("""
SELECT
    date_trunc('month', tpep_pickup_datetime) AS hour,
    PULocationID AS zone,
    

    SUM(total_amount) AS amount,

    -- # records in each zone
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    -- filter out data not from 2020
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [12]:
df_yellow_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 00:00:00| 258| 10078.70999999999|           291|
|2020-01-01 00:00:00| 191|21641.960000000003|           472|
|2020-01-01 00:00:00|  36|14542.690000000024|           575|
|2020-01-01 00:00:00| 171| 8920.479999999994|           247|
|2020-01-01 00:00:00|  59|352.81999999999994|            10|
|2020-01-01 00:00:00|  53| 7636.119999999998|           198|
|2020-01-01 00:00:00| 190|3243.4200000000014|           160|
|2020-01-01 00:00:00| 259| 7821.699999999998|           200|
|2020-01-01 00:00:00|   2|             76.69|             3|
|2020-01-01 00:00:00| 114|1305615.4400008079|         78908|
|2020-01-01 00:00:00| 208|13712.950000000023|           360|
|2020-01-01 00:00:00| 124| 13568.84000000002|           318|
|2020-01-01 00:00:00| 187|            680.47|             8|
|2020-01-01 00:00:00|  4

In [13]:
df_yellow_revenue.repartition(20).write.parquet('data/report/revenue/yellow', mode='overwrite')

In [14]:
# instead of computing everything on the fly, we'll use materialized results that were saved before
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [15]:
# to differentiate between green/yellow column names
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amt') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amt') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [16]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [17]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|         green_amt|green_number_records|        yellow_amt|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|   1|            466.61|                   4| 74478.65000000053|                  755|
|2020-01-01 00:00:00|   2|              null|                null|             76.69|                    3|
|2020-01-01 00:00:00|   3| 14820.07999999998|                 510| 7942.369999999995|                  206|
|2020-01-01 00:00:00|   4|           2972.79|                 100|181770.73999999434|                10147|
|2020-01-01 00:00:00|   5|              null|                null|2387.8000000000006|                   39|
|2020-01-01 00:00:00|   6|            198.18|                   4|            595.36|                   12|
|2020-01-01 00:00:00|   7|20

In [18]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [19]:
df_join = spark.read.parquet('data/report/revenue/total')

In [20]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|         green_amt|green_number_records|        yellow_amt|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|   1|            466.61|                   4| 74478.65000000053|                  755|
|2020-01-01 00:00:00|   2|              null|                null|             76.69|                    3|
|2020-01-01 00:00:00|   3| 14820.07999999998|                 510| 7942.369999999995|                  206|
|2020-01-01 00:00:00|   4|           2972.79|                 100|181770.73999999434|                10147|
|2020-01-01 00:00:00|   5|              null|                null|2387.8000000000006|                   39|
|2020-01-01 00:00:00|   6|            198.18|                   4|            595.36|                   12|
|2020-01-01 00:00:00|   7|20

In [23]:
df_zones = spark.read.parquet('zones/')

In [24]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [27]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones', mode='overwrite')